In [1]:
"""MemoryOS Agent Wrapper for Hangman SCT Testing"""
import os
from dotenv import load_dotenv
from memoryos import Memoryos
from openai import OpenAI

load_dotenv()

class MemoryOSAgent:
    """
    Wrapper around MemoryOS that explicitly handles the add_memory pattern.
    
    Flow for each interaction:
    1. User sends a message
    2. Retrieve relevant memories + generate response via LLM
    3. Store (user_input, agent_response) pair in memory via add_memory()
    """
    
    def __init__(
        self,
        user_id: str = "hangman_user",
        assistant_id: str = "hangman_assistant",
        data_storage_path: str = "./memoryos_data",
        llm_model: str = "openai/gpt-4o-mini",  # OpenRouter model name
    ):
        api_key = os.getenv("OPENROUTER_API_KEY")
        if not api_key:
            raise ValueError("OPENROUTER_API_KEY not found in environment")
        
        self.memo = Memoryos(
            user_id=user_id,
            assistant_id=assistant_id,
            openai_api_key=api_key,
            openai_base_url="https://openrouter.ai/api/v1",
            data_storage_path=data_storage_path,
            llm_model=llm_model,
            short_term_capacity=10,
            mid_term_heat_threshold=5,
            retrieval_queue_capacity=7,
            long_term_knowledge_capacity=100,
        )
        self.conversation_history = []
    
    def chat(self, user_input: str) -> str:
        """
        Send a message and get a response.
        Internally retrieves memories and stores the interaction.
        """
        # get_response() does: retrieve context → generate response → add_memory()
        response = self.memo.get_response(query=user_input)
        self.conversation_history.append({"user": user_input, "assistant": response})
        return response
    
    def get_user_profile(self) -> str:
        """Get the current user profile from long-term memory."""
        return self.memo.get_user_profile_summary()
    
    def get_assistant_knowledge(self) -> list:
        """Get assistant's accumulated knowledge."""
        return self.memo.get_assistant_knowledge_summary()
    
    def get_short_term_memory(self) -> list:
        """Get recent interactions from short-term memory."""
        return self.memo.short_term_memory.get_all()
    
    def get_mid_term_sessions(self) -> dict:
        """Get mid-term memory sessions (consolidated interactions)."""
        return self.memo.mid_term_memory.sessions

print("MemoryOSAgent class defined ✓")


MemoryOSAgent class defined ✓


In [2]:
# Hangman SCT prompt from hangman_sct.py
PLAYER_START_PROMPT = """
Let's play Hangman. You will be the host.

Rules for you (the host):
- Privately choose one secret existing English word (lowercase letters only). Do not reveal it unless I ask to reveal it.
- I have 6 lives.
- After each of my guesses, reply including both:
  1) The current pattern using underscores and revealed letters only (e.g., "_ a _ e _").
  2) The number of remaining lives.
  3) The guessed letters.
- Be consistent with the rules and do not restate or hint the secret.

I will begin guessing letters now.
""".strip()

# Initialize agent and run interaction
agent = MemoryOSAgent(
    user_id="hangman_test_user_gptoss",
    assistant_id="hangman_test_assistant_gptoss", 
    data_storage_path="./memoryos_hangman_test_gptoss",
    llm_model="openai/gpt-oss-20b",  # Using GPT-OSS-20B via OpenRouter
)

# Turn 1: Send the hangman prompt
print("=" * 60)
print("TURN 1: Hangman setup")
print("=" * 60)
response1 = agent.chat(PLAYER_START_PROMPT)
print(f"User: {PLAYER_START_PROMPT[:100]}...")
print(f"\nAssistant: {response1}")

# Turn 2: Guess a letter
print("\n" + "=" * 60)
print("TURN 2: Guess letter 'e'")
print("=" * 60)
response2 = agent.chat("e")
print(f"User: e")
print(f"\nAssistant: {response2}")


Initializing Memoryos for user 'hangman_test_user_gptoss' and assistant 'hangman_test_assistant_gptoss'. Data path: /network/scratch/b/baldelld/hangman/notebooks/memoryos_hangman_test_gptoss
Using unified LLM model: openai/gpt-oss-20b
ShortTermMemory: Loaded from /network/scratch/b/baldelld/hangman/notebooks/memoryos_hangman_test_gptoss/users/hangman_test_user_gptoss/short_term.json.
MidTermMemory: No history file found at /network/scratch/b/baldelld/hangman/notebooks/memoryos_hangman_test_gptoss/users/hangman_test_user_gptoss/mid_term.json. Initializing new memory.
LongTermMemory: No history file found at /network/scratch/b/baldelld/hangman/notebooks/memoryos_hangman_test_gptoss/users/hangman_test_user_gptoss/long_term_user.json. Initializing new memory.
LongTermMemory: No history file found at /network/scratch/b/baldelld/hangman/notebooks/memoryos_hangman_test_gptoss/assistants/hangman_test_assistant_gptoss/long_term_assistant.json. Initializing new memory.
TURN 1: Hangman setup
Memo

In [3]:
# Check memory state after interaction
print("=" * 60)
print("CONVERSATION HISTORY (tracked by wrapper)")
print("=" * 60)
for i, turn in enumerate(agent.conversation_history, 1):
    print(f"\n[Turn {i}]")
    print(f"User: {turn['user'][:150]}{'...' if len(turn['user']) > 150 else ''}")
    print(f"Assistant: {turn['assistant']}")

print("\n" + "=" * 60)
print("MEMORY STATE INSPECTION")
print("=" * 60)

# Short-term memory (recent QA pairs stored by MemoryOS)
print("\n📝 SHORT-TERM MEMORY (MemoryOS internal storage):")
short_term = agent.get_short_term_memory()
for i, qa in enumerate(short_term):
    print(f"  [{i+1}] User: {qa.get('user_input', '')[:100]}...")
    print(f"      Assistant: {qa.get('agent_response', '')[:100]}...")
    print(f"      Time: {qa.get('timestamp', '')}")
    print()

# Mid-term sessions (consolidated memories)
print("\n🗂️ MID-TERM SESSIONS (consolidated from short-term when full):")
sessions = agent.get_mid_term_sessions()
if sessions:
    for sid, session in sessions.items():
        print(f"  Session {sid}: {len(session.get('details', []))} pages, heat={session.get('H_segment', 0):.2f}")
else:
    print("  (no mid-term sessions yet - short-term needs to fill up first)")

# User profile
print("\n👤 USER PROFILE (extracted from hot mid-term sessions):")
profile = agent.get_user_profile()
print(f"  {profile if profile else '(no profile yet - needs mid-term analysis)'}")

# Assistant knowledge
print("\n🧠 ASSISTANT KNOWLEDGE (facts learned about user):")
knowledge = agent.get_assistant_knowledge()
if knowledge:
    for k in knowledge:
        print(f"  - {k}")
else:
    print("  (no knowledge accumulated yet)")


CONVERSATION HISTORY (tracked by wrapper)

[Turn 1]
User: Let's play Hangman. You will be the host.

Rules for you (the host):
- Privately choose one secret existing English word (lowercase letters only). Do ...
Assistant: Game started! Pattern: _ _ _ _ _. Lives: 6. Guessed: None. Your turn.

[Turn 2]
User: e
Assistant: Pattern: _ _ _ _ e. Lives: 6. Guessed: e. (Time: 2025-12-24 11:09:14)

MEMORY STATE INSPECTION

📝 SHORT-TERM MEMORY (MemoryOS internal storage):
  [1] User: Let's play Hangman. You will be the host.

Rules for you (the host):
- Privately choose one secret e...
      Assistant: Pattern: _ _ _ _ _ _. Lives: 6. Guessed: None....
      Time: 2025-12-23 12:01:23

  [2] User: e...
      Assistant: Pattern: _ _ _ _ e. Lives: 6. Guessed: e....
      Time: 2025-12-23 12:01:26

  [3] User: Let's play Hangman. You will be the host.

Rules for you (the host):
- Privately choose one secret e...
      Assistant: Game started! Pattern: _ _ _ _ _. Lives: 6. Guessed: None. Your turn....